In [ ]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import torch
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install git+https://github.com/facebookresearch/segment-anything.git

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-78c8wq42
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-78c8wq42
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done


In [ ]:
def show_anns(anns, axes=None, image_file=None,  save_flag=False):
    saveImg_dir = "/content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask"
    if len(anns) == 0:
        return
    if axes:
        ax = axes
    else:
        ax = plt.gca()
        ax.set_autoscale_on(False)
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)

    for i, ann in enumerate(sorted_anns):
        m = ann['segmentation']

        mask_img = np.zeros((m.shape[0], m.shape[1], 3), dtype=np.uint8)

        mask_img[m > 0] = [255, 255, 255]
        print("image_file : ", image_file)

        if image_file and i == 1 and save_flag:
            file_name = os.path.basename(os.path.splitext(image_file)[0])
            file_path = os.path.join(saveImg_dir, f"{file_name}_mask_{i}.bmp")
            print("file_path : ", file_path)
            cv2.imwrite(file_path, mask_img)

        for i in range(3):
            img = np.ones((m.shape[0], m.shape[1], 3))
            color_mask = np.random.random((1, 3)).tolist()[0]
            img[:,:,i] = color_mask[i]
            ax.imshow(np.dstack((img, m*0.5)))

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)


def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)


def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))


In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "/content/drive/MyDrive/Colab Notebooks/pretrain_model/sam_vit_b_01ec64.pth"
model_type = "vit_b"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam, points_per_batch=16)

In [ ]:
image_dir = '/content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask'

image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]

for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    image_array = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    masks = mask_generator.generate(image_array)

    _, axes = plt.subplots(1, 3, figsize=(16, 16))
    axes[0].imshow(image_array)
    show_anns(masks, axes[1], image_path, True)
    axes[2].imshow(image_array)
    show_anns(masks, axes[2], None, False)

print("Automatically Generating Object Masks with SAM END!!")



image_file :  /content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask/012.png
image_file :  /content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask/012.png
file_name :  012
file_path :  /content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask/012_mask_1.bmp
image_file :  /content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask/012.png
image_file :  /content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask/012.png
image_file :  /content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask/012.png
image_file :  /content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask/012.png
image_file :  /content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask/012.png
image_file :  /content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask/012.png
image_file :  /content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask/012.png
image_file :  /content/drive/MyDrive/Colab Notebooks/sample_img/hazelnut_mask/012.png
image_file :  /content/drive/My